In [4]:
#importar bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import time
import random
import re

In [5]:
# Lista de URL para recolha de dados

#https://portuguesecharts.com/archive.asp?sparte=a&woche=1&jahr=2004&todo=show

static = 'https://portuguesecharts.com/archive.asp?sparte=a&woche='
static2 = '&jahr='
static3 = '&todo=show'

urllist = [static+str(week)+static2+str(year)+static3 for year in range (2003,2022) for week in range(1,53)]

## Ciclo de recolha de dados

In [6]:
maindata = pd.DataFrame()

#urllist_test = urllist[23:25]

for i in tqdm(urllist):
    headers = requests.utils.default_headers()
    headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
    URL = i
    year = str(re.findall(r'(\d+)(?!.*\d)', URL))
    cal_week = str(re.findall(r'^\D*(\d+)', URL))
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.content, "lxml")
    
    chart_table = [name.text for name in soup.find_all("td", attrs = {'class':'text'})] 
    chart_table  = chart_table[1:] #remove first useless thing
    
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
    
    chart = list(chunks(chart_table, 7))
    #print(chart)
    chart_data = pd.DataFrame(chart)
    chart_data.reset_index(inplace=True)
    
    chart_data["calendar_week"] = cal_week
    chart_data["year"] = year
    maindata = pd.concat([maindata, chart_data])
    time.sleep(random.randint(1,2))
    
print("Já fomos lá buscar tudo")


Já fomos lá buscar tudo


## Limpar os dados para exportação

In [7]:
maindata.dtypes

index            object
0                object
1                object
2                object
3                object
4                object
5                object
6                object
calendar_week    object
year             object
dtype: object

In [8]:
maindata.shape

(35730, 10)

In [9]:
maindata.rename(columns = {0: "rank", 1:"previous_week", 2: "empty", 3:"artist", 4: "album", 5:"label", 6:"weeks_on_chart"}, inplace=True)
maindata

,index,rank,previous_week,empty,artist,album,label,weeks_on_chart,calendar_week,year
0,0,1,,,Adiafa,Adiafa,Columbia,1,['1'],['2003']
1,1,2,,,Operação Triunfo,Operação Triunfo gala 3,Ariola,1,['1'],['2003']
2,2,3,(2),,Robbie Williams,Escapology,Chrysalis,17,['1'],['2003']
3,3,4,(13),,Operação Triunfo,Operação Triunfo,Ariola,2,['1'],['2003']
4,4,5,(3),,Soundtrack,8 Mile,Interscope,9,['1'],['2003']
...,...,...,...,...,...,...,...,...,...,...
45,45,46,(49),,Tony Carreira,As canções das nossas vidas,Sony Music,111,['52'],['2021']
46,46,47,(26),,Ana Moura,Moura,Universal,179,['52'],['2021']
47,47,48,(24),,Linkin Park,Hybrid Theory,Warner,18,['52'],['2021']
48,48,49,,,Il Divo,For Once In My Life: A Celebration Of Motown,Universal,1,['52'],['2021']


In [10]:
maindata.drop(['empty', "index"], axis=1, inplace = True)

In [11]:
maindata['previous_week'] = maindata['previous_week'].str.strip("()")
maindata

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year
0,1,,Adiafa,Adiafa,Columbia,1,['1'],['2003']
1,2,,Operação Triunfo,Operação Triunfo gala 3,Ariola,1,['1'],['2003']
2,3,2,Robbie Williams,Escapology,Chrysalis,17,['1'],['2003']
3,4,13,Operação Triunfo,Operação Triunfo,Ariola,2,['1'],['2003']
4,5,3,Soundtrack,8 Mile,Interscope,9,['1'],['2003']
...,...,...,...,...,...,...,...,...
45,46,49,Tony Carreira,As canções das nossas vidas,Sony Music,111,['52'],['2021']
46,47,26,Ana Moura,Moura,Universal,179,['52'],['2021']
47,48,24,Linkin Park,Hybrid Theory,Warner,18,['52'],['2021']
48,49,,Il Divo,For Once In My Life: A Celebration Of Motown,Universal,1,['52'],['2021']


In [12]:
maindata['calendar_week'] = maindata['calendar_week'].str.strip("['']")
maindata["year"] = maindata["year"].str.strip("['']")
maindata

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year
0,1,,Adiafa,Adiafa,Columbia,1,1,2003
1,2,,Operação Triunfo,Operação Triunfo gala 3,Ariola,1,1,2003
2,3,2,Robbie Williams,Escapology,Chrysalis,17,1,2003
3,4,13,Operação Triunfo,Operação Triunfo,Ariola,2,1,2003
4,5,3,Soundtrack,8 Mile,Interscope,9,1,2003
...,...,...,...,...,...,...,...,...
45,46,49,Tony Carreira,As canções das nossas vidas,Sony Music,111,52,2021
46,47,26,Ana Moura,Moura,Universal,179,52,2021
47,48,24,Linkin Park,Hybrid Theory,Warner,18,52,2021
48,49,,Il Divo,For Once In My Life: A Celebration Of Motown,Universal,1,52,2021


In [13]:
maindata = maindata.astype({'rank': 'int64', 'weeks_on_chart': 'int64', "calendar_week" : 'int64', "year": 'int64'}).dtypes
maindata.to_csv("charts30-50.csv", index=False)